In [34]:
import pandas as pd
import os

In [35]:
parquet_folder_path = '/home/manuto/Documents/world_bank/bert_twitter_labor/code/twitter/code/2-twitter_labor/model_training/saved_data'
may20_data_path = '/home/manuto/Documents/world_bank/bert_twitter_labor/code/twitter/data/may20_9Klabels/data_binary_pos_neg_balanced'
may28_data_path = '/home/manuto/Documents/world_bank/bert_twitter_labor/code/twitter/data/may28_iter1'

In [36]:
def convert_labels_to_int(label):
    if label == 'yes':
        return 1
    elif label == 'no':
        return 0

In [37]:
new_labels_df = pd.read_parquet(os.path.join(parquet_folder_path, 'training_data1.parquet'))
new_labels_df.columns = ['tweet_id', 'text', 'sel_sample', 'sel_class', 'sel_threshold', 'model_score',
                                     'model',
                                     'Unnamed: 0', 'is_unemployed', 'lost_job_1mo', 'job_search', 'is_hired_1mo',
                                     'job_offer']

In [38]:
new_labels_df.head()

,tweet_id,text,sel_sample,sel_class,sel_threshold,model_score,model,Unnamed: 0,is_unemployed,lost_job_1mo,job_search,is_hired_1mo,job_offer
0,323595079333863424,Yall quit.talking about my baby Kobe !!!!! ):,filtered,is_unemployed,0.9,0.900345,BERT,628,no,no,no,no,no
1,376093437813993473,@OpTicIRed @RawCFGR i got laid off from my job...,filtered,is_unemployed,0.9,0.900337,BERT,1920,no,no,no,yes,no
2,577363004150386688,Keep that album to ya self and just quit now @...,filtered,is_unemployed,0.9,0.900333,BERT,5477,no,no,no,no,no
3,735169610853408771,omg I got a job interview today,filtered,is_unemployed,0.9,0.900331,BERT,8390,no,no,yes,no,no
4,578373483081678848,@socarolinesays I got fired from a job once an...,filtered,is_unemployed,0.9,0.900330,BERT,5492,no,no,no,no,no


In [39]:
for label in ["lost_job_1mo", "is_unemployed", "job_search", "is_hired_1mo", "job_offer"]:
    #import former train and eval data
    train_df = pd.read_csv(os.path.join(may20_data_path, "train_{}.csv".format(label)))
    eval_df = pd.read_csv(os.path.join(may20_data_path, "val_{}.csv".format(label)))
    train_df = train_df[["text", "class"]]
    eval_df = eval_df[["text", "class"]]
    #rename columns
    train_df.columns = ['text', 'labels']
    eval_df.columns = ['text', 'labels']
    #format new label,ed data
    new_labels_df_2 = new_labels_df[['text', label]]
    new_labels_df_2.columns = ['text', 'labels']
    #convert labels to int and drop unsure
    new_labels_df_2['labels'] = new_labels_df_2['labels'].apply(convert_labels_to_int)
    new_labels_df_2 = new_labels_df_2[new_labels_df_2['labels'].notna()].reset_index(drop=True)
    #concat all data
    all_data_df = pd.concat([train_df, eval_df, new_labels_df_2]).reset_index(drop=True)
    # 70/30 split
    train_df = all_data_df.sample(frac=0.7, random_state=0)
    eval_df = all_data_df.drop(train_df.index)
    train_df = train_df.reset_index(drop=True)
    eval_df = eval_df.reset_index(drop=True)
    
    print('**********', label, '**********')
    print('Combined train shape: ', train_df.shape[0])
    print('Combined eval shape: ', eval_df.shape[0])
    print(eval_df.head())
    train_df.to_csv(os.path.join(may28_data_path, 'train_{}.csv'.format(label)), index=False)
    eval_df.to_csv(os.path.join(may28_data_path, 'val_{}.csv'.format(label)), index=False)

********** lost_job_1mo **********
Combined train shape:  1919
Combined eval shape:  823
                                                text  labels
0  RT @thedailybeast: 30 percent of Glenn Beck’s ...     0.0
1  Here is a system that allows you to sell other...     0.0
2  Today was a long ass day. Woke up at 5:30 went...     0.0
3  I recently had a dream of missing my return fl...     0.0
4  Job scam tried to give me an offer today via t...     0.0
********** is_unemployed **********
Combined train shape:  3219
Combined eval shape:  1380
                                                text  labels
0  I was late again got work today. They gonna fi...     0.0
1  Now the owners wife is having a breakdown and ...     0.0
2  Now I have to do illegal shit to obtain this m...     1.0
3  Laid Off Now they want me back. - https://t.co...     1.0
4  Now Im quitting McDonalds altogether.  󾌦 https...     0.0
********** job_search **********
Combined train shape:  2196
Combined eval shape:  941
 

/home/manuto/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
